In [ ]:
from multiprocessing import Pool
import time
import numpy as np
import psutil
from sklearn.metrics import silhouette_score
from greedy_clustering import greedy_clustering, euclidean


REP_FILE = 'representatives.csv'
ASGNMT_FILE = 'assignments.csv'
K_MEANS_CENTROID_FILE = 'kmeans_centroids.csv'
K_MEANS_LABELS_FILE = 'kmeans_labels.csv'
FINAL_ASSGMNT_GREEDY_FILE = 'final_a1.csv'
FINAL_ASSGMNT_SUBSAMPLING_FILE = 'final_a2.csv'


def subsampling(file_path, ratio, seed):
    """
    Baseline A2 approach: randomly subsample 'ratio' fraction of the data
    to form the reduced set R. Saves R to representatives.csv
    """
    print("Reading full data for subsampling...")
    data = np.loadtxt(file_path, delimiter=',')   # shape (N, d)
    rng = np.random.default_rng(seed=seed)

    # Number of points to sample
    sample_size = int(ratio * len(data))

    # Pick 'sample_size' distinct random indices
    indices = rng.choice(len(data), size=sample_size, replace=False)
    representatives = data[indices]               # shape (sample_size, d)

    # Save to the same file name used by the greedy approach, for uniformity
    np.savetxt(REP_FILE, representatives, delimiter=",")
    print(f"Subsampled R saved to {REP_FILE} with shape {representatives.shape}")
    return None

### TODO: Add/Change functions below
def _assign_chunk(chunk, centroids):
    """
    Helper for parallel assignment step in K-Means.
    Returns a list of cluster labels (nearest centroid) for each point in 'chunk'.
    """
    labels_chunk = []
    for x in chunk:
        dists = [euclidean(x, c) for c in centroids]
        labels_chunk.append(np.argmin(dists))
    return labels_chunk

# Helper function to monitor memory usage
def monitor_memory():
    process = psutil.Process()
    return process.memory_info().rss / (1024 * 1024)  # Memory in MB

# Evaluate clustering quality
def evaluate_clustering(assignments_file, centroid_file, data_file):
    assignments = np.loadtxt(assignments_file, delimiter=",")
    centroids = np.loadtxt(centroid_file, delimiter=",")
    data = np.loadtxt(data_file, delimiter=",")

    # Silhouette Score
    labels = assignments[:, -1]  # Last column contains cluster labels
    silhouette = silhouette_score(data, labels)
    
    # Compute intra-cluster and inter-cluster distances
    intra_distances = []
    for i, centroid in enumerate(centroids):
        points = data[labels == i]
        if len(points) > 0:
            intra_distances.append(np.linalg.norm(points - centroid, axis=1).mean())
    intra_distance = np.mean(intra_distances)

    inter_distances = [
        np.linalg.norm(centroids[i] - centroids[j])
        for i in range(len(centroids)) for j in range(i + 1, len(centroids))
    ]
    inter_distance = np.mean(inter_distances)

    return {
        "silhouette_score": silhouette,
        "intra_cluster_distance": intra_distance,
        "inter_cluster_distance": inter_distance
    }

# Function to run comparisons for A1 and A2
def compare_methods(file_path):
    metrics = {}
    clustering_results = {}

    # Run A1 (Greedy Clustering)
    print("Comparing A1 (Greedy Clustering)...")
    start_time = time.time()
    start_mem = monitor_memory()
    tau = 100
    greedy_clustering(file_path, tau, seed=1, nr_workers=4)
    k_means(REP_FILE, K=3, max_iters=100, seed=1)
    end_time = time.time()
    end_mem = monitor_memory()

    metrics['A1'] = {
        "time": end_time - start_time,
        "memory": end_mem - start_mem
    }

    clustering_results['A1'] = evaluate_clustering(
        assignments_file=FINAL_ASSGMNT_GREEDY_FILE,
        centroid_file=K_MEANS_CENTROID_FILE,
        data_file=file_path
    )

    # Run A2 (Subsampling)
    print("Comparing A2 (Subsampling)...")
    start_time = time.time()
    start_mem = monitor_memory()
    ratio = 0.10  # Example: 10% of data
    subsampling(file_path, ratio=ratio, seed=1)
    k_means(REP_FILE, K=3, max_iters=100, seed=1)
    end_time = time.time()
    end_mem = monitor_memory()

    metrics['A2'] = {
        "time": end_time - start_time,
        "memory": end_mem - start_mem
    }

    clustering_results['A2'] = evaluate_clustering(
        assignments_file=FINAL_ASSGMNT_SUBSAMPLING_FILE,
        centroid_file=K_MEANS_CENTROID_FILE,
        data_file=file_path
    )

    # Print results
    print("\nResource Usage Comparison:")
    for method, usage in metrics.items():
        print(f"Method: {method}")
        print(f"  Time: {usage['time']} seconds")
        print(f"  Memory: {usage['memory']} MB")

    print("\nClustering Quality Metrics:")
    for method, results in clustering_results.items():
        print(f"Method: {method}")
        print(f"  Silhouette Score: {results['silhouette_score']}")
        print(f"  Intra-cluster Distance: {results['intra_cluster_distance']}")
        print(f"  Inter-cluster Distance: {results['inter_cluster_distance']}")

### TODO: Add/Change functions above

def k_means(R_file, K, max_iters, seed):
    """
    Perform K-Means on the reduced dataset R (read from R_file).
    Saves final centroids and labels:
        - kmeans_centroids.csv  (shape: (K, d))
        - kmeans_labels.csv     (shape: (|R|,))
    The order of labels corresponds to the order of points in R.
    """
    print("Reading representatives for K-Means...")
    nr_workers = 4
    R = np.loadtxt(R_file, delimiter=",")  # shape: (|R|, d)
    rng = np.random.default_rng(seed=seed)
    N, d = R.shape

    # Initialize K centroids by picking K random points from R
    idx = rng.choice(N, size=K, replace=False)
    centroids = R[idx].copy()  # shape: (K, d)

    # Initialize cluster labels array
    labels = np.zeros(N, dtype=int)

    # K-Means loop
    for iteration in range(max_iters):
        # E-Step: assign each point in R to nearest centroid
        if nr_workers > 1:
            # Parallel approach
            chunks = np.array_split(R, nr_workers, axis=0)
            with Pool(processes=nr_workers) as pool:
                partial_labels = pool.starmap(_assign_chunk, [(chunk, centroids) 
                                                              for chunk in chunks])
            # Flatten the partial labels back
            labels = np.concatenate([np.array(lbls) for lbls in partial_labels])
        else:
            # Single-thread approach
            for i in range(N):
                dists = [euclidean(R[i], c) for c in centroids]
                labels[i] = np.argmin(dists)

        # M-Step: recompute each centroid as mean of assigned points
        new_centroids = np.zeros_like(centroids)
        for k in range(K):
            points_k = R[labels == k]
            if len(points_k) > 0:
                new_centroids[k] = np.mean(points_k, axis=0)
            else:
                # If no points assigned to cluster k, re-initialize randomly
                new_centroids[k] = R[rng.integers(N)]

        # Check for convergence
        if np.allclose(new_centroids, centroids):
            print(f"Converged at iteration {iteration}")
            centroids = new_centroids
            break
        centroids = new_centroids

    # Save results
    np.savetxt(K_MEANS_CENTROID_FILE, centroids, delimiter=",")
    np.savetxt(K_MEANS_LABELS_FILE, labels, delimiter=",")
    print(f"K-Means finished. Saved centroids to {K_MEANS_CENTROID_FILE} and labels to {K_MEANS_LABELS_FILE}.")
    return None


def remap_labels(kmeans_labels_file, kmeans_centroids_file, assignments_file, dataset_path, subsample):
    if not subsample:
        ### NOTE: assignments shape is (num_data_points, 7)
        assignments = np.loadtxt(assignments_file, delimiter=",")
    else:
        ### NOTE: assignments shape is (num_data_points, 6)
        ### NOTE: since the subsampling doesn't save the assignments, it is only the datapoints.
        assignments = np.loadtxt(dataset_path, delimiter=",")
    
    kmeans_centroids = np.loadtxt(kmeans_centroids_file, delimiter=",")
    
    ### NOTE: kmeans_labels has length num_representatives
    kmeans_labels = np.loadtxt(kmeans_labels_file, delimiter=",")

    ### NOTE: X_labels has length num_data_points
    X_labels = np.zeros(len(assignments), dtype=int)
    ### NOTE: final_assignments has shape (num_data_points,7) , 6 dimensions for data points and 1 dimension for cluster label of kmeans
    final_assignments = np.zeros((len(assignments), 7))

    ### TODO: Insert code below
    if not subsample:
        N = len(assignments)
        for i in range(N):
            # Which representative was this point assigned to?
            rep_index = int(assignments[i, 6])
            # That rep_index maps to kmeans_labels[rep_index]
            cluster_label = kmeans_labels[rep_index]
            # Fill the final assignments
            final_assignments[i, :6] = assignments[i, :6]  # original data
            final_assignments[i, 6] = cluster_label
    else:
        X = np.loadtxt(dataset_path, delimiter=",")  # shape (N, d)
        R = np.loadtxt(REP_FILE, delimiter=",")      # shape (|R|, d)
        N = len(X)
        for i, x in enumerate(X):
            # Find nearest subsampled representative
            dists = [euclidean(x, r) for r in R]
            nearest_rep = np.argmin(dists)
            cluster_label = kmeans_labels[nearest_rep]
            final_assignments[i, :6] = x
            final_assignments[i, 6] = cluster_label
    ### TODO: Insert code above

    if subsample:
        np.savetxt(FINAL_ASSGMNT_SUBSAMPLING_FILE, final_assignments, delimiter=",")
    else:
        np.savetxt(FINAL_ASSGMNT_GREEDY_FILE, final_assignments, delimiter=",")

    return None

if __name__ == "__main__":
    file_path = "small_dataset.csv"
    subsample = True

    if subsample:
        # A2: Subsampling
        ratio = 0.10   # ex 10% of data
        subsampling(file_path, ratio=ratio, seed=1)
    else:
        # A1: Greedy clustering
        tau = 100
        greedy_clustering(file_path, tau, seed=1, nr_workers=4)

    # Run K-Means on the reduced data in representatives.csv
    K = 3
    max_iters = 100
    k_means(REP_FILE, K=K, max_iters=max_iters, seed=1)

    # Postprocessing: map cluster labels back to the full dataset
    # For greedy approach => it uses assignments.csv
    # For subsampling    => it reads directly from file_path
    remap_labels(
        kmeans_labels_file=K_MEANS_LABELS_FILE,
        kmeans_centroids_file=K_MEANS_CENTROID_FILE,
        assignments_file=ASGNMT_FILE,
        dataset_path=file_path,
        subsample=subsample
    )

    # Run comparison after main functionalities
    print("\nStarting A1 vs A2 comparison...")
    compare_methods(file_path)
    print("Done.")
